In [7]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import sqrt


In [8]:
blazepose_json_dir = Path("outputs/blazepose_mediapipe/2")
baseline_json_dir = Path("outputs/simple_landmarks/2/json")
blazepose_overlay_dir = Path("outputs/blazepose_mediapipe/2")
frames_dir = Path("frames/frames2")


In [9]:
def draw_keypoints(image, points, color, label_prefix=""):
    for i, (x, y) in enumerate(points):
        if x > 0 and y > 0:
            cv2.circle(image, (int(x), int(y)), 5, color, -1)
            cv2.putText(image, f"{label_prefix}{i+1}", (int(x)+5, int(y)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)


In [10]:
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


In [ ]:
key_order = [
    "nose", "left_eye", "right_eye", "thumb",
    "pointer_finger", "middle_finger", "ring_finger", "pinky_finger"
]

mediapipe_to_baseline = {
    "nose": 0,
    "left_eye": 2,
    "right_eye": 5,
    "thumb": 22,
    "pointer_finger": 20,
    "middle_finger": 16,
    "ring_finger": 18,
    "pinky_finger": 18
}


In [ ]:
# Load BlazePose MediaPipe results
blazepose_json_path = blazepose_json_dir / "blazepose_landmarks.json"

if not blazepose_json_path.exists():
    blazepose_data = {}
else:
    with open(blazepose_json_path, "r") as f:
        blazepose_data = json.load(f)


In [ ]:
all_frame_errors = []

for i in range(1, 7):
    frame_name = f"frame_{i:02d}.jpg"
    baseline_path = baseline_json_dir / f"frame_{i:02d}.json"

    if not baseline_path.exists():
        continue
    
    blazepose_entry = None
    for key in blazepose_data.keys():
        if frame_name in key or f"frame_{i:02d}" in key:
            blazepose_entry = blazepose_data[key]
            break
    
    if not blazepose_entry or len(blazepose_entry) == 0:
        continue
    
    detection = blazepose_entry[0]
    keypoints = detection.get("keypoints", [])
    
    if len(keypoints) == 0:
        continue
    
    # Convert keypoints from [x, y, v, x, y, v, ...] format to list of (x, y) tuples
    # MediaPipe has 33 landmarks, so 33 * 3 = 99 values
    mediapipe_points_all = [(keypoints[j], keypoints[j+1]) for j in range(0, len(keypoints), 3)]
    
    # Extract relevant keypoints matching baseline
    blazepose_points = []
    for k in key_order:
        idx = mediapipe_to_baseline.get(k, -1)
        if idx >= 0 and idx < len(mediapipe_points_all):
            blazepose_points.append(mediapipe_points_all[idx])
        else:
            blazepose_points.append((0, 0))
    
    # Load baseline data
    with open(baseline_path, "r") as f:
        baseline_data = json.load(f)
    baseline_points = [(baseline_data[k]["coordinates"]["x"], baseline_data[k]["coordinates"]["y"]) for k in key_order]
    
    # Load image
    img_path = frames_dir / frame_name
    if not img_path.exists():
        overlay_path = blazepose_overlay_dir / f"{Path(frame_name).stem}_blazepose.jpg"
        if overlay_path.exists():
            img_path = overlay_path
        else:
            continue
    
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    draw_keypoints(img, baseline_points, (0, 255, 0), "B")
    draw_keypoints(img, blazepose_points, (255, 0, 0), "M")
    
    errors = []
    for b, m in zip(baseline_points, blazepose_points):
        dist = euclidean_distance(b, m)
        errors.append(dist)
        cv2.line(img, (int(b[0]), int(b[1])), (int(m[0]), int(m[1])), (255, 255, 0), 1)
    
    mean_error = np.mean(errors)
    max_error = np.max(errors)
    all_frame_errors.append(mean_error)
    
    # Display results
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"{frame_name}: Mean error={mean_error:.2f}px | Max={max_error:.2f}px")
    plt.axis("off")
    plt.show()
    
    print(f"\nFrame {i:02d} Results")
    for k, dist in zip(key_order, errors):
        print(f"  {k:15s}: {dist:7.2f}px")
    print(f"  Mean error: {mean_error:.2f}px | Max error: {max_error:.2f}px")

if all_frame_errors:
    print(f"Average mean error across frames: {np.mean(all_frame_errors):.2f}px")
    print(f"Best (lowest) frame mean error: {np.min(all_frame_errors):.2f}px")
    print(f"Worst (highest) frame mean error: {np.max(all_frame_errors):.2f}px")
